# Modération de contenu

**IMPORTANT** : Créez une ressource *Content Moderator* (ne pas confondre avec *AI Content Safety*) et renseignez `SERVEUR` avec votre point de terminaison (endpoint). Récupérez également la clé et renseignez là lors de l'exécution de la première cellule.

In [3]:
from getpass import getpass

CLE = getpass('Clé')
SERVEUR = 'https://moder00.cognitiveservices.azure.com/'

# Ce produit est nul...
TEXTE = "Ce porduit est nul, ils m'ont pris pour un idiot. Contactez-moi sur mon mail : spam@syllab.com"


## Simple

In [4]:
import requests

reponse = requests.post(
  url = f'{SERVEUR}/contentmoderator/moderate/v1.0/ProcessText/Screen',
  headers = {
    'Ocp-Apim-Subscription-Key': CLE,
    'Content-Type': 'text/plain; charset=utf-8'
  },
  params = {'autocorrect': True, 'PII': True},
  data = TEXTE
)
reponse.json()

{'OriginalText': "Ce porduit est nul, ils m'ont pris pour un idiot. Contactez-moi sur mon mail : spam@syllab.com",
 'NormalizedText': "Ce porduit est nul, ils m'ont pris pour  idiot. Contactez-moi sur mon mail : spam@syllab.com",
 'AutoCorrectedText': "Ce produit est nul, ils m'ont pris pour un idiot. Contactez-moi sur mon mail : spam@syllab.com",
 'Misrepresentation': None,
 'PII': {'Email': [{'Detected': 'spam@syllab.com',
    'SubType': 'Regular',
    'Text': 'spam@syllab.com',
    'Index': 79}],
  'IPA': [],
  'Phone': [],
  'Address': [],
  'SSN': []},
 'Language': 'fra',
 'Terms': None,
 'Status': {'Code': 3000, 'Description': 'OK', 'Exception': None},
 'TrackingId': 'ca23706d-3438-4b5e-bdf8-9cba85aabd3f'}

## Création d'un liste de modération

In [5]:
reponse = requests.post(
  url = f'{SERVEUR}/contentmoderator/lists/v1.0/termlists',
  headers = {
    'Ocp-Apim-Subscription-Key': CLE,
    'Content-Type': 'application/json'
  },
  json={
    "Name": "Boutique",
    "Description": "Liste des termes sensibles dans les commentaires de la boutique",
    "Metadata": {
      "Projet": "Boutique",
      "Role": "public"
    }
  }
)
resultat = reponse.json()
listId = resultat['Id']
resultat

{'Id': 113,
 'Name': 'Boutique',
 'Description': 'Liste des termes sensibles dans les commentaires de la boutique',
 'Metadata': {'Projet': 'Boutique', 'Role': 'public'}}

## Énumération des listes

In [6]:
reponse = requests.get(
    url = f'{SERVEUR}/contentmoderator/lists/v1.0/termlists/',
    headers = { 'Ocp-Apim-Subscription-Key': CLE }
)
reponse.json()

[{'Id': 113,
  'Name': 'Boutique',
  'Description': 'Liste des termes sensibles dans les commentaires de la boutique',
  'Metadata': {'Projet': 'Boutique', 'Role': 'public'}}]

## Ajout de termes dans la liste

In [7]:
import time

ATTENTE_OFFRE_ESSAI = 2

for terme in ['bidon', 'arnaque', 'KONK']:
  reponse = requests.post(
    url = f'{SERVEUR}/contentmoderator/lists/v1.0/termlists/{listId}/terms/{terme}',
    params = {'language': 'fra'},
    headers = {'Ocp-Apim-Subscription-Key': CLE}
  )
  print(reponse)
  time.sleep(ATTENTE_OFFRE_ESSAI)


<Response [201]>
<Response [201]>
<Response [201]>


## Rafraîchissement de l'index de la liste

In [8]:
reponse = requests.post(
    url = f'{SERVEUR}/contentmoderator/lists/v1.0/termlists/{listId}/RefreshIndex',
    params = { 'language': 'fra' },
    headers = { 'Ocp-Apim-Subscription-Key': CLE }
)
reponse.json()

{'ContentSourceId': '113',
 'IsUpdateSuccess': True,
 'AdvancedInfo': [],
 'Status': {'Code': 3000, 'Description': 'OK', 'Exception': None},
 'TrackingId': 'b961fea1-5aba-4b41-bad0-ad0226bfcac2'}

## Recherche de termes avec liste personnalisée

In [10]:
listId = reponse.json()['ContentSourceId']

In [11]:
reponse = requests.post(
  url = f'{SERVEUR}/contentmoderator/moderate/v1.0/ProcessText/Screen',
  headers={
    'Ocp-Apim-Subscription-Key': CLE,
    'Content-Type': 'text/plain; charset=utf-8'
  },
  params={
    'autocorrect': True,
    'PII': True,
    'listId': listId
  },
  data = "N'y allez pas c'est une arnaque. Allez plutôt chez KONK."
)

reponse.json()

{'OriginalText': "N'y allez pas c'est une arnaque. Allez plutôt chez KONK",
 'NormalizedText': "N'y allez pas c'est une arnaque. Allez plutôt chez KONK",
 'AutoCorrectedText': "N'y allez pas c'est une arnaque. Allez plutôt chez KONK",
 'Misrepresentation': None,
 'Language': 'fra',
 'Terms': [{'Index': 24,
   'OriginalIndex': 24,
   'ListId': 113,
   'Term': 'arnaque'},
  {'Index': 51, 'OriginalIndex': 51, 'ListId': 113, 'Term': 'KONK'}],
 'Status': {'Code': 3000, 'Description': 'OK', 'Exception': None},
 'TrackingId': '0b1945c3-2048-4c7d-bc3a-85f476be5ff5'}